In [1]:
from transformers import AutoTokenizer, AutoModel
import torch
!pip install pypdf
from pypdf import PdfReader
import re
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Load BioClinicalBERT model and tokenizer
model_name = "emilyalsentzer/Bio_ClinicalBERT"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Parse PDF document and split into sentences
def parse_document(file_path):
    reader = PdfReader(file_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text() + " "
    sentences = re.split(r'(?<=[.!?]) +', text)
    return sentences

# Get CLS token embedding for each sentence
def get_sentence_embedding(sentence):
    inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    cls_embedding = outputs.last_hidden_state[:, 0, :]  # CLS token embedding
    return cls_embedding.squeeze().numpy()

# Read PDF and generate embeddings for each sentence
file_path = "/content/sample.pdf"  # Replace with the path to your PDF file
sentences = parse_document(file_path)
embeddings = [get_sentence_embedding(sentence) for sentence in sentences]

# Display results
print("Total sentences:", len(sentences))
print("Embedding of the first sentence:", embeddings[0])
print("Embedding shape:", embeddings[0].shape)  # Should be (768,)

# Define the query and get its embedding
query = "How is the patient’s sensory function across various lumbar and sacral segments?"  # Replace this with your actual query sentence
query_embedding = get_sentence_embedding(query)  # Ensure query_embedding has shape (768,)

# Calculate cosine similarities
similarities = cosine_similarity([query_embedding], embeddings)[0]

# Define minimum and maximum thresholds
initial_threshold = 0.5
max_threshold = 10
step = 0.01  # Increment step for threshold adjustment

# Initialize variables to store the best response and threshold
best_response = None
best_threshold = initial_threshold

threshold = initial_threshold
while threshold <= max_threshold:
    response = None  # Reset response for each threshold

    # Check if there's any sentence with similarity above the current threshold
    for i, sim in enumerate(similarities):
        if sim > threshold:
            response = f"{sentences[i]}"
            break  # Found a response, so break the inner loop

    # If a response was found, store it as the current best and increase the threshold
    if response:
        best_response = response
        best_threshold = threshold
        threshold += step  # Try the next higher threshold
    else:
        # No response found at the current threshold, stop the loop
        break

# Output the best result found at the highest valid threshold
if best_response:
    print(f"Best response: '{best_response}' at threshold: {best_threshold}")
else:
    print("No similar sentences found.")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 11.1 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

Total sentences: 8
Embedding of the first sentence: [ 2.23955423e-01 -2.85251558e-01 -4.89268117e-02  2.90236045e-02
 -3.39991182e-01  2.01563567e-01  5.12752950e-01 -2.32126251e-01
  2.45374382e-01 -5.29578188e-03 -4.82457459e-01  1.14766806e-01
 -2.86699831e-01 -2.07805578e-02 -8.55050907e-02  2.17180073e-01
  9.48838145e-02  1.91758633e-01 -6.45845294e-01 -4.32666242e-02
 -5.21423519e-01  2.54255384e-01 -7.50821233e-02 -7.08196238e-02
  9.79949832e-02  5.10677136e-02 -1.60437019e-03  4.27234828e-01
  3.02044809e-01  6.18283153e-01  2.82012880e-01  2.31449887e-01
 -4.38455731e-01 -7.47469962e-02 -9.92579460e-02  3.50342631e-01
  8.30245088e-04  2.35692829e-01 -4.19190824e-01 -7.63225481e-02
 -2.52932534e-02 -1.20007031e-01  1.01933670e+00  4.96886447e-02
  3.47981453e-01 -4.19901788e-01  2.98917145e-01  5.25315523e-01
 -6.35398984e-01  3.66517603e-01  2.04717577e-01  3.34309846e-01
 -5.23287244e-03  1.51460096e-02  1.75952613e-01 -2.41286919e-01
 -2.93966919e-01 -2.61019051e-01  1.57